In [ ]:
import argparse
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'

import torch
import numpy as np
import random
import wandb
import models
import yaml
from trainer import Trainer
import pprint
from torchvision import transforms
import datasets


np.random.seed(0)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

# Load YAML config
with open("./configs/noisycifar10/train_noisycifar10_ce_symm40.yaml", 'r') as file:
    config = yaml.safe_load(file)

# # read YAML file from string
# config = yaml.safe_load(
# """
# data:
#   dataset: noisy_cifar10
#   noise_rate: 0.4
#   noise_type: symmetric
#   random_seed: 42

# model:
#   architecture: resnet18
#   num_classes: 10

# wandb:
#   mode: disabled # "disabled" or "online"
#   entity: siit-iitp
#   project: noisy-label

# trainer:
#   optimizer: sgd
#   init_lr: 1.0e-1
#   momentum: 0.9
#   weight_decay: 1.0e-4
#   lr_scheduler: multistep
#   max_epoch: 200
#   loss_fn: cross_entropy
#   num_classes: 10
#   num_workers: 2
#   batch_size: 128
#   save_model: true
# """
# )

model = models.get_model(**config["model"]).cuda()
model = torch.jit.script(model)


WANDB_RUN_ID = "mquy2drg" # NoisyCIFAR10(symm,0.4)-CE
# WANDB_RUN_ID = "ccnf390c" # NoisyCIFAR10(symm,0.4)-MAE
# WANDB_RUN_ID = "i0qx1u8n" # NoisyCIFAR10(symm,0.4)-CE-AutoAugment


trainer = Trainer(
                model=model,
                config=config['trainer'],
                )


def load_checkpoint(name="model_199.pth"):
    checkpoint = wandb.restore(name, run_path=f"siit-iitp/noisy-label/{WANDB_RUN_ID}", replace=True, root='./temp')
    trainer.model.load_state_dict(torch.load(checkpoint.name, map_location="cuda"))
    print(f"Loaded checkpoint: {checkpoint.name}")

print(trainer.criterion)




auto_augment = transforms_v2.AutoAugment(AutoAugmentPolicy.CIFAR10)
random_crop = transforms.Compose([
    transforms_v2.RandomCrop(32, padding=4),
    transforms_v2.RandomHorizontalFlip(),
])
num_augment = 100

transform_train = transforms.Compose([
    transforms.Lambda(lambda x: torch.tensor(np.array(x)).permute(2,0,1)),
    transforms.Lambda(lambda x: torch.stack([auto_augment(x) for _ in range(num_augment)] + [random_crop(x) for _ in range(num_augment)], dim=0)), # (11, C, H, W)
])


train_dataset, _ = datasets.get_dataset(**config["data"])
train_dataset.transform = transform_train

config["trainer"]["num_workers"] = 32
config["trainer"]["batch_size"] = 200
dataloader = trainer.get_dataloader(train_dataset, train=False)

sampled_dataset = {
    'image': [],
    'target': [],
    'target_gt': [],
}
for batch in tqdm.tqdm(dataloader):
    sampled_dataset['image'].append(batch['image'])
    sampled_dataset['target'].append(batch['target'])
    sampled_dataset['target_gt'].append(batch['target_gt'])
sampled_dataset = {k: torch.cat(v, dim=0) for k, v in sampled_dataset.items()}





import torch.utils.data
train_dataset = torch.utils.data.TensorDataset(*list(sampled_dataset.values()))
dataloader = trainer.get_dataloader(train_dataset, train=False)

normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010), inplace=True)
dp_model = torch.nn.DataParallel(trainer.model)

def run_inference(i):
    load_checkpoint(name=f"model_{i}.pth")

    with torch.no_grad():
        trainer.model.eval()
        result = {
            'output': [],
            'target': [],
            'target_gt': [],
        }
        for batch in tqdm.tqdm(dataloader):
            data = batch[0].cuda().float().div_(255.0)
            target = batch[1].cuda()
            target_gt = batch[2].cuda() # (B,)

            b = data.size(0)
            data = normalize(data)
            with torch.cuda.amp.autocast():
                output = dp_model(data.view(-1,3,32,32)) # (B*11, 10)
            result['output'].append(output.view(b, -1, 10))
            result['target'].append(target)
            result['target_gt'].append(target_gt)

        result = {k: torch.cat(v, dim=0).cpu() for k, v in result.items()}

    import pandas as pd
    df = pd.DataFrame({
        'autoaugment': torch.unbind(result['output'][:,:num_augment,:]),
        'randomcrop': torch.unbind(result['output'][:,num_augment:,:]),
        'is_noisy': (result['target'] != result['target_gt']).tolist(), # (50000, 11, 10)
        'target': result['target'].tolist(), # (50000,)
        'target_gt': result['target_gt'].tolist(), # (50000,)
        })
    torch.save(df, f'./model_{i}_result.pt')


for k in range(9, 200, 10):
    run_inference(k)

In [3]:
import argparse
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'

import torch
import numpy as np
import random
import wandb
import models
import yaml
from trainer import Trainer
import pprint
from torchvision import transforms
import datasets
import torchvision.transforms.v2 as transforms_v2
from torchvision.transforms import AutoAugmentPolicy
import tqdm.auto as tqdm
import einops


np.random.seed(0)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

# Load YAML config
with open("./configs/noisycifar10/train_noisycifar10_ce_symm40.yaml", 'r') as file:
    config = yaml.safe_load(file)

# # read YAML file from string
# config = yaml.safe_load(
# """
# data:
#   dataset: noisy_cifar10
#   noise_rate: 0.4
#   noise_type: symmetric
#   random_seed: 42

# model:
#   architecture: resnet18
#   num_classes: 10

# wandb:
#   mode: disabled # "disabled" or "online"
#   entity: siit-iitp
#   project: noisy-label

# trainer:
#   optimizer: sgd
#   init_lr: 1.0e-1
#   momentum: 0.9
#   weight_decay: 1.0e-4
#   lr_scheduler: multistep
#   max_epoch: 200
#   loss_fn: cross_entropy
#   num_classes: 10
#   num_workers: 2
#   batch_size: 128
#   save_model: true
# """
# )

model = models.get_model(**config["model"])
model.fc = torch.nn.Identity()
model = model.cuda()
model = torch.jit.script(model)


WANDB_RUN_ID = "mquy2drg" # NoisyCIFAR10(symm,0.4)-CE
# WANDB_RUN_ID = "ccnf390c" # NoisyCIFAR10(symm,0.4)-MAE
# WANDB_RUN_ID = "i0qx1u8n" # NoisyCIFAR10(symm,0.4)-CE-AutoAugment


trainer = Trainer(
                model=model,
                config=config['trainer'],
                )


def load_checkpoint(name="model_199.pth"):
    checkpoint = wandb.restore(name, run_path=f"siit-iitp/noisy-label/{WANDB_RUN_ID}", replace=True, root='./temp')
    trainer.model.load_state_dict(torch.load(checkpoint.name, map_location="cuda"), strict=False)
    print(f"Loaded checkpoint: {checkpoint.name}")

print(trainer.criterion)




auto_augment = transforms_v2.AutoAugment(AutoAugmentPolicy.CIFAR10)
random_crop = transforms.Compose([
    transforms_v2.RandomCrop(32, padding=4),
    transforms_v2.RandomHorizontalFlip(),
])
num_augment = 100

transform_train = transforms.Compose([
    transforms.Lambda(lambda x: torch.tensor(np.array(x)).permute(2,0,1)),
    transforms.Lambda(lambda x: torch.stack([auto_augment(x) for _ in range(num_augment)] + [random_crop(x) for _ in range(num_augment)], dim=0)), # (11, C, H, W)
    transforms.Lambda(lambda x: x.float().div_(255.0)),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010), inplace=True),
])


train_dataset, _ = datasets.get_dataset(**config["data"])
train_dataset.transform = transform_train

config["trainer"]["num_workers"] = 8
config["trainer"]["batch_size"] = 200
dataloader = trainer.get_dataloader(train_dataset, train=False)


dp_model = torch.nn.DataParallel(trainer.model)

load_checkpoint(name=f"model_199.pth")

with torch.no_grad():
    trainer.model.eval()
    result = {
        'output': [],
        'target': [],
        'target_gt': [],
    }
    for batch in tqdm.tqdm(dataloader):
        data = batch['image'].cuda()
        target = batch['target']
        target_gt = batch['target_gt'] # (B,)

        b = data.size(0)
        with torch.cuda.amp.autocast():
            output = dp_model(einops.rearrange(data, 'b n c h w -> (b n) c h w')) # (B*11, 10)
        result['output'].append(einops.rearrange(output, '(b n) c -> b n c', n=2*num_augment).cpu().float())
        result['target'].append(target)
        result['target_gt'].append(target_gt)

    result = {k: torch.cat(v, dim=0).cpu() for k, v in result.items()}

# import pandas as pd
# df = pd.DataFrame({
#     'autoaugment': torch.unbind(result['output'][:,:num_augment,:]),
#     'randomcrop': torch.unbind(result['output'][:,num_augment:,:]),
#     'is_noisy': (result['target'] != result['target_gt']).tolist(), # (50000, 11, 10)
#     'target': result['target'].tolist(), # (50000,)
#     'target_gt': result['target_gt'].tolist(), # (50000,)
#     })

CrossEntropyLoss()
Files already downloaded and verified
Loaded checkpoint: ./temp/model_199.pth


  0%|          | 0/250 [00:00<?, ?it/s]

In [8]:
torch.save(result, f'./embddings.pt')

In [1]:
import torch

result = torch.load('./embddings.pt')

In [2]:
num_augment=100
temp = {
    'autoaugment': result['output'][:,:num_augment,:],
    'randomcrop': result['output'][:,num_augment:,:],
    'is_noisy': result['target'] != result['target_gt'], # (50000, 11, 10)
    'target': result['target'], # (50000,)
    'target_gt': result['target_gt'], # (50000,)
}

In [4]:
torch.save(temp, f'./embeddings.pt')

In [5]:
temp = torch.load('./embeddings.pt')

In [9]:

import pandas as pd
df = pd.DataFrame({
    'autoaugment': torch.unbind(result['output'][:,:num_augment,:]),
    'randomcrop': torch.unbind(result['output'][:,num_augment:,:]),
    'is_noisy': (result['target'] != result['target_gt']).tolist(), # (50000, 11, 10)
    'target': result['target'].tolist(), # (50000,)
    'target_gt': result['target_gt'].tolist(), # (50000,)
    })

In [2]:
result['output'].shape

torch.Size([50000, 10240, 10])

In [6]:
df.iloc[0,0].shape

torch.Size([100, 10])

## Figure 3

In [ ]:
import numpy as np
import torch
import numpy.random as nprn
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
import numpy as np
from scipy.special import softmax

df = torch.load('./figure3.pt')
df

In [ ]:
df_clean = df[df['is_noisy']==False]
df_noisy = df[df['is_noisy']==True]



def get_projection_matrix(k):
    theta = (2*np.pi/k) * (np.arange(k) + 0.5)
    return np.stack([np.sin(theta), np.cos(theta)], axis=0)

num_classes = 10 # CIFAR-10
T = get_projection_matrix(num_classes)


cifar10_classes = ['airplane$\mathcal{T}$', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
fig = plt.figure(figsize=(35,20))
for idx in range(28):
    row = df_noisy.iloc[idx]
    # row = df_clean.iloc[idx]
    blue = row['autoaugment_logits'].float().div(1).softmax(-1).numpy()
    # pointpred = row['pointpred'][None, :].softmax(-1).numpy()
    red = row['randomcrop_logits'].float().div(1).softmax(-1).numpy()

    blue = T @ blue.T
    red = T @ red.T

    ax = fig.add_subplot(4,7,idx + 1)

    # Plot the octagon
    vertices = T @ np.eye(num_classes)
    vertices = np.hstack([vertices, vertices[:, 0:1]])
    for i, vertex in enumerate(vertices[:,:-1].T):
        ax.text(vertex[0], vertex[1], cifar10_classes[i], ha='center', va='center', fontsize=12)
        ax.plot([0, vertex[0]], [0, vertex[1]], 'k--', linewidth=1)

    # Plot the points
    ax.scatter(blue[0], blue[1], linewidth=0, alpha=0.3, zorder=10)
    ax.scatter([red[0]], [red[1]], color='red', alpha = 0.3, s=50, linewidth=0, zorder=12)

    # Set the axis limits
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.set_aspect('equal')
    ax.set_title(f"target: {cifar10_classes[row['target']]}, gt: {cifar10_classes[row['target_gt']]}")
    ax.set_axis_off()

# Show the plot
plt.tight_layout()
plt.savefig('figure3.pdf')
plt.show()

## Figure 6

In [ ]:
import matplotlib.pyplot as plt

def plot_ece(conf, correct, bin_width=0.1):

    bins = []
    bin_width = 0.1
    ece = 0.0
    for low in np.arange(0.0, 1.0, bin_width):
        mask = (confidence >= low) & (confidence < low + bin_width)
        acc = correct[mask].float().mean().item()
        bins.append(acc)
        if mask.sum() > 0:
            ece += np.abs(acc - (low + bin_width/2)) * mask.sum()
    bins = np.nan_to_num(bins)
    ece = ece / len(conf)

    fig, ax = plt.subplots()
    # ax.bar([f"{x:.2f}" for x in np.arange(0.0, 1.0, bin_width)], bins)
    ax.bar(np.arange(0.0, 1.0, bin_width), bins, align='edge', width=0.1)
    print(bins)
    ax.plot([0, 1], [0, 1], color='red', linestyle='--')
    ax.set_title(f"{ece=:.2f}")

    # set the xlim to (0,1)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_xlabel('Confidence')
    ax.set_ylabel('Accuracy')
    ax.set_aspect('equal')
    plt.tight_layout()
    plt.savefig('figure6.pdf')

    plt.show()


# correct = torch.stack(tuple(df['logits']), dim=0).float().softmax(-1).argmax(-1) == torch.tensor(df['target_gt'])
# confidence = torch.stack(tuple(df['logits']), dim=0).float().softmax(-1).max(-1).values
# plot_ece(confidence, correct)

# correct = torch.stack(tuple(df['randomcrop_logits']), dim=0).float().softmax(-1).mean(1).argmax(-1) == torch.tensor(df['target_gt'])
# confidence = torch.stack(tuple(df['randomcrop_logits']), dim=0).float().softmax(-1).mean(1).max(-1).values
# plot_ece(confidence, correct)

correct = torch.stack(tuple(df['autoaugment_logits']), dim=0).float().softmax(-1).mean(1).argmax(-1) == torch.tensor(df['target_gt'])
confidence = torch.stack(tuple(df['autoaugment_logits']), dim=0).float().softmax(-1).mean(1).max(-1).values
plot_ece(confidence, correct)

## Figure 5

- https://wandb.ai/siit-iitp/noisy-label_baselines/runs/auc5c96t
- https://wandb.ai/siit-iitp/noisy-label_baselines/runs/kapwliv4

## Figure 7-(a)

- https://wandb.ai/siit-iitp/noisy-label/runs/i0qx1u8n
- https://wandb.ai/siit-iitp/noisy-label/runs/niuft3hk

## Figure 7-(b)

- https://wandb.ai/siit-iitp/noisy-label/runs/uv2cklac
- https://wandb.ai/siit-iitp/noisy-label/runs/2bd0sdnd
- https://wandb.ai/siit-iitp/noisy-label/runs/niuft3hk
- https://wandb.ai/siit-iitp/noisy-label/runs/mquy2drg

## Figure 8

- https://wandb.ai/siit-iitp/noisy-label/runs/h4hlrxf9
- https://wandb.ai/siit-iitp/noisy-label/runs/k1cilssr
- https://wandb.ai/siit-iitp/noisy-label/runs/uv2cklac
- https://wandb.ai/siit-iitp/noisy-label/runs/2bd0sdnd
- https://wandb.ai/siit-iitp/noisy-label/runs/mquy2drg